<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/%ED%81%B4%EB%A6%B0%EB%A3%B8_HVAC_%EC%B5%9C%EC%A0%81%ED%99%94_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_(%EA%B0%9C%EC%84%A0%EB%90%9C_%EB%8D%94%EB%AF%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B0%8F_%EC%B5%9C%EC%A0%81%ED%99%94).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

# --- 0. 유틸리티 함수 (엔탈피 계산) ---
def calculate_humidity_ratio(temp_c, rel_hum_percent, p_atm_pa=101325):
    if pd.isna(temp_c) or pd.isna(rel_hum_percent) or rel_hum_percent < 0 or rel_hum_percent > 100:
        return np.nan
    p_ws_pa = 611.21 * np.exp((18.678 - temp_c / 234.5) * (temp_c / (257.14 + temp_c)))
    p_w_pa = (rel_hum_percent / 100) * p_ws_pa
    if p_atm_pa <= p_w_pa: # 대기압보다 수증기압이 높거나 같으면 계산 불가
        return np.nan
    humidity_ratio = 0.621945 * p_w_pa / (p_atm_pa - p_w_pa)
    return humidity_ratio

def calculate_enthalpy(temp_c, humidity_ratio):
    if pd.isna(temp_c) or pd.isna(humidity_ratio): return np.nan
    enthalpy = 1.006 * temp_c + humidity_ratio * (2501 + 1.86 * temp_c)
    return enthalpy

class CleanroomHVACOptimizer:
    def __init__(self, num_oacs=14, num_low_chillers=8, num_high_chillers=8, num_cooling_towers=9):
        self.num_oacs = num_oacs
        self.num_low_chillers = num_low_chillers
        self.num_high_chillers = num_high_chillers
        self.num_cooling_towers = num_cooling_towers

        self.oac_scalers_X = {}
        self.oac_models = {}
        self.oac_feature_names = {}
        self.oac_target_names = {}

        self.low_ch_scaler_X = None
        self.low_ch_model = None
        self.low_ch_feature_names = []
        self.low_ch_target_names = []

        self.high_ch_scaler_X = None
        self.high_ch_model = None
        self.high_ch_feature_names = []
        self.high_ch_target_names = []

        # 최적화를 위한 설비 대당 "정격" 또는 "최대 효율" 용량
        # 이 값들은 실제 시스템의 설계값 또는 검증된 운전 데이터를 기반으로 설정해야 합니다.
        # 단위: 이전에 사용된 '부하 지표(코일 개도율 합계 등)'와 동일하게 사용
        self.rated_capacity_per_low_chiller = 700  # 예시: 저온 냉동기 1대당 정격 부하 처리 능력
        self.rated_capacity_per_high_chiller = 800 # 예시: 고온 냉동기 1대당 정격 부하 처리 능력
        self.rated_heat_rejection_per_ct = 1200 # 예시: 냉각탑 1대당 정격 열 방출 능력

        # 최적 운전 부하율 범위 (예: 40% ~ 85%)
        self.optimal_load_ratio_min = 0.40
        self.optimal_load_ratio_max = 0.85


    def _prepare_oac_data_single(self, df_full, oac_id_num):
        if '외기_엔탈피' not in df_full.columns:
            df_full['외기_습도비'] = df_full.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
            df_full['외기_엔탈피'] = df_full.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)

        oac_col_prefix = f"OAC{oac_id_num}_"
        features = [
            '외기_온도', '외기_습도', '외기_엔탈피',
            f'{oac_col_prefix}예열_set_point', f'{oac_col_prefix}예냉_set_point',
            f'{oac_col_prefix}냉각_set_point', f'{oac_col_prefix}승온_set_point',
            f'{oac_col_prefix}가습_set_point', f'{oac_col_prefix}FAN_Hz'
        ]
        targets = [
            f'{oac_col_prefix}예열_개도율', f'{oac_col_prefix}예냉_개도율',
            f'{oac_col_prefix}냉각_개도율', f'{oac_col_prefix}승온_개도율',
            f'{oac_col_prefix}가습_개도율'
        ]
        valid_features = [f for f in features if f in df_full.columns]
        valid_targets = [t for t in targets if t in df_full.columns]

        if not valid_features or len(valid_features) != len(features) or \
           not valid_targets or len(valid_targets) != len(targets):
            # print(f"경고: OAC {oac_id_num} 데이터 준비 중 컬럼 부족. 요구 피처 수: {len(features)}, 발견: {len(valid_features)}. 요구 타겟 수: {len(targets)}, 발견: {len(valid_targets)}")
            return None, None, [], []

        X = df_full[valid_features].copy()
        y = df_full[valid_targets].copy()
        X.fillna(X.median(), inplace=True) # 평균 대신 중앙값 사용 고려
        y.fillna(y.median(), inplace=True)
        return X, y, valid_features, valid_targets

    def train_oac_models(self, df_historical_data):
        print("OAC 모델 학습 시작...")
        temp_df = df_historical_data.copy()
        if '외기_엔탈피' not in temp_df.columns: # 함수 호출 전 미리 계산
            temp_df['외기_습도비'] = temp_df.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
            temp_df['외기_엔탈피'] = temp_df.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)

        for i in range(1, self.num_oacs + 1):
            X_oac, y_oac, features, targets = self._prepare_oac_data_single(temp_df, i)
            if X_oac is None:
                print(f"  OAC {i} 모델 학습 건너<0xEB><0><0x84>니다 (데이터 준비 실패).")
                continue
            self.oac_feature_names[i] = features
            self.oac_target_names[i] = targets
            scaler = StandardScaler()
            X_oac_scaled = scaler.fit_transform(X_oac)
            self.oac_scalers_X[i] = scaler
            X_train, _, y_train, _ = train_test_split(X_oac_scaled, y_oac, test_size=0.2, shuffle=False, random_state=42)
            model = MultiOutputRegressor(RandomForestRegressor(n_estimators=60, random_state=42, n_jobs=-1, max_depth=18, min_samples_leaf=4, min_samples_split=5)) # 파라미터 약간 조정
            model.fit(X_train, y_train)
            self.oac_models[i] = model
        print("OAC 모델 학습 완료.")

    def predict_oac_coil_opening_ratios(self, current_outdoor_conditions_dict, oac_setpoints_list_of_dicts, modified_coil_info=None):
        all_oac_predictions = {}
        outdoor_temp = current_outdoor_conditions_dict['외기_온도']
        outdoor_humidity = current_outdoor_conditions_dict['외기_습도']
        outdoor_hr = calculate_humidity_ratio(outdoor_temp, outdoor_humidity)
        outdoor_enthalpy = calculate_enthalpy(outdoor_temp, outdoor_hr)
        current_outdoor_df_dict = {'외기_온도': outdoor_temp, '외기_습도': outdoor_humidity, '외기_엔탈피': outdoor_enthalpy}

        for i in range(1, self.num_oacs + 1):
            if i not in self.oac_models or i not in self.oac_scalers_X or not self.oac_feature_names.get(i):
                target_names_for_oac = self.oac_target_names.get(i, [f"OAC{i}_코일{j}_개도율" for j in range(1,6)])
                all_oac_predictions[i] = {name: np.nan for name in target_names_for_oac}
                continue

            current_oac_input_dict = current_outdoor_df_dict.copy()
            if i-1 < len(oac_setpoints_list_of_dicts):
                 current_oac_input_dict.update(oac_setpoints_list_of_dicts[i-1])

            input_data_values = [current_oac_input_dict.get(fn, np.nan) for fn in self.oac_feature_names[i]]
            input_df = pd.DataFrame([input_data_values], columns=self.oac_feature_names[i])
            input_df.fillna(input_df.median(), inplace=True) # 중앙값으로 채우기

            input_scaled = self.oac_scalers_X[i].transform(input_df)
            prediction_array = self.oac_models[i].predict(input_scaled)
            oac_pred_dict = dict(zip(self.oac_target_names[i], prediction_array[0]))
            all_oac_predictions[i] = oac_pred_dict

        if modified_coil_info:
            mod_oac_id = modified_coil_info['oac_id_num']
            mod_coil_name = modified_coil_info['coil_target_name']
            mod_value = modified_coil_info['value']
            if mod_oac_id in all_oac_predictions and mod_coil_name in all_oac_predictions[mod_oac_id]:
                original_value = all_oac_predictions[mod_oac_id][mod_coil_name]
                all_oac_predictions[mod_oac_id][mod_coil_name] = mod_value
                print(f"알림: OAC {mod_oac_id}의 {mod_coil_name} 개도율이 {original_value:.2f}%에서 {mod_value:.2f}%(으)로 수동 변경되었습니다.")
                print("      주의: 이 변경으로 인한 다른 코일들의 자동 연동 조정은 현재 단순 덮어쓰기 방식입니다.")
            else:
                print(f"경고: 수정하려는 코일 정보(OAC ID: {mod_oac_id}, 코일명: {mod_coil_name})를 찾을 수 없습니다.")
        return all_oac_predictions

    def _prepare_chiller_data_common(self, df_full, chiller_type):
        temp_df = df_full.copy()
        if '외기_엔탈피' not in temp_df.columns and '외기_온도' in temp_df.columns and '외기_습도' in temp_df.columns:
             temp_df['외기_습도비'] = temp_df.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
             temp_df['외기_엔탈피'] = temp_df.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)

        relevant_coil_suffix = '_예냉_개도율' if chiller_type == 'high' else '_냉각_개도율'
        total_relevant_coil_opening = pd.Series(0.0, index=temp_df.index)
        for oac_id in range(1, self.num_oacs + 1):
            oac_col_prefix = f"OAC{oac_id}_"
            coil_col_name = f"{oac_col_prefix}{relevant_coil_suffix.lstrip('_')}"
            if coil_col_name in temp_df.columns:
                fan_hz_col = f"{oac_col_prefix}FAN_Hz"
                weight = temp_df[fan_hz_col].fillna(temp_df[fan_hz_col].median()) if fan_hz_col in temp_df.columns else 1
                total_relevant_coil_opening += temp_df[coil_col_name].fillna(0) * weight
        temp_df[f'Total_{chiller_type}_Coil_Load_Indicator'] = total_relevant_coil_opening

        features = [f'Total_{chiller_type}_Coil_Load_Indicator', '외기_온도', '외기_습도', '외기_엔탈피']
        targets_map = {
            'low': ['저온냉동기_메인차압압력', '저온냉동기_메인차압개도율', '저온냉동기_서브차압개도율'],
            'high': ['고온냉동기_메인차압압력', '고온냉동기_메인차압개도율', '고온냉동기_서브차압개도율']
        }
        targets = targets_map[chiller_type]
        valid_features = [f for f in features if f in temp_df.columns]
        valid_targets = [t for t in targets if t in temp_df.columns]

        if not valid_features or len(valid_features) != len(features) or \
           not valid_targets or len(valid_targets) != len(targets):
            # print(f"경고: {chiller_type} 냉동기 데이터 준비 중 컬럼 부족.")
            return None, None, [], []

        X = temp_df[valid_features].copy()
        y = temp_df[valid_targets].copy()
        X.fillna(X.median(), inplace=True)
        y.fillna(y.median(), inplace=True)
        return X, y, valid_features, valid_targets

    def train_chiller_models(self, df_historical_data):
        print("냉동기 모델 학습 시작...")
        temp_hist_data = df_historical_data.copy()
        if '외기_엔탈피' not in temp_hist_data.columns: # 함수 호출 전 미리 계산
            temp_hist_data['외기_습도비'] = temp_hist_data.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
            temp_hist_data['외기_엔탈피'] = temp_hist_data.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)

        for chiller_type, model_attr_prefix in [('low', 'low_ch'), ('high', 'high_ch')]:
            # print(f"  {chiller_type.capitalize()} 온도 냉동기 모델 학습 중...")
            X_ch, y_ch, f_ch, t_ch = self._prepare_chiller_data_common(temp_hist_data, chiller_type)
            if X_ch is None:
                print(f"  {chiller_type.capitalize()} 온도 냉동기 모델 학습 건너<0xEB><0><0x84>니다.")
                continue

            setattr(self, f"{model_attr_prefix}_feature_names", f_ch)
            setattr(self, f"{model_attr_prefix}_target_names", t_ch)
            scaler = StandardScaler().fit(X_ch)
            setattr(self, f"{model_attr_prefix}_scaler_X", scaler)
            X_ch_scaled = scaler.transform(X_ch)
            X_train, _, y_train, _ = train_test_split(X_ch_scaled, y_ch, test_size=0.2, shuffle=False, random_state=42)
            model = MultiOutputRegressor(RandomForestRegressor(n_estimators=60, random_state=42, n_jobs=-1, max_depth=12, min_samples_leaf=5)) # 파라미터 약간 조정
            model.fit(X_train, y_train)
            setattr(self, f"{model_attr_prefix}_model", model)
        print("냉동기 모델 학습 완료.")

    def predict_chiller_parameters(self, all_oac_coil_predictions_dict, current_outdoor_conditions_dict):
        chiller_predictions_output = {'low_temp_chiller': {}, 'high_temp_chiller': {}}
        outdoor_temp = current_outdoor_conditions_dict['외기_온도']
        outdoor_humidity = current_outdoor_conditions_dict['외기_습도']
        outdoor_hr = calculate_humidity_ratio(outdoor_temp, outdoor_humidity)
        outdoor_enthalpy = calculate_enthalpy(outdoor_temp, outdoor_hr)
        current_outdoor_plus_enthalpy = {**current_outdoor_conditions_dict, '외기_엔탈피': outdoor_enthalpy}

        for chiller_type, model_attr_prefix in [('low', 'low_ch'), ('high', 'high_ch')]:
            model = getattr(self, f"{model_attr_prefix}_model")
            scaler_X = getattr(self, f"{model_attr_prefix}_scaler_X")
            feature_names = getattr(self, f"{model_attr_prefix}_feature_names")
            target_names = getattr(self, f"{model_attr_prefix}_target_names")

            if not model or not scaler_X or not feature_names:
                chiller_predictions_output[f"{chiller_type}_temp_chiller"] = {tn: np.nan for tn in target_names}
                chiller_predictions_output[f"{chiller_type}_temp_chiller"]['calculated_total_load_indicator'] = np.nan
                continue

            total_load_indicator = 0
            coil_suffix_keyword = '냉각_개도율' if chiller_type == 'low' else '예냉_개도율'

            # FAN_Hz 값을 oac_setpoints_list_of_dicts 에서 가져와야 하나, 이 함수는 해당 인자를 받지 않음.
            # 학습 시에는 데이터프레임에서 가져왔으므로, 예측 시에도 일관성을 위해 FAN_Hz 정보를 어딘가에서 가져와야 함.
            # 여기서는 단순화를 위해 FAN_Hz 가중치를 1로 가정하거나, all_oac_coil_predictions_dict 에 FAN_Hz가 포함되어 있다고 가정해야 함.
            # 지금은 FAN_Hz 가중치 없이 단순 합산으로 변경 (또는 예측 입력 시 FAN_Hz를 OAC setpoint와 함께 전달받도록 구조 변경 필요)
            for oac_id in range(1, self.num_oacs + 1):
                oac_preds = all_oac_coil_predictions_dict.get(oac_id, {})
                relevant_coil_key = next((k for k in oac_preds.keys() if coil_suffix_keyword in k), None)
                if relevant_coil_key:
                    # fan_hz_val = 1 # 임시로 FAN_Hz 가중치 1로 설정
                    # To-Do: oac_setpoints_list_of_dicts를 이 함수에 전달받아 FAN_Hz를 가져오거나,
                    # all_oac_coil_predictions_dict 생성 시 FAN_Hz 정보를 포함시켜야 함.
                    # 현재는 oac_setpoints_list_of_dicts가 없으므로 가중치 없이 합산.
                    total_load_indicator += oac_preds.get(relevant_coil_key, 0)


            input_data = {f'Total_{chiller_type}_Coil_Load_Indicator': total_load_indicator, **current_outdoor_plus_enthalpy}
            input_df_list = [input_data.get(fn, np.nan) for fn in feature_names]
            input_df = pd.DataFrame([input_df_list], columns=feature_names).fillna(input_df.median()) # 중앙값으로 채우기

            input_df_scaled = scaler_X.transform(input_df)
            pred_array = model.predict(input_df_scaled)

            chiller_predictions_output[f'{chiller_type}_temp_chiller'] = dict(zip(target_names, pred_array[0]))
            chiller_predictions_output[f'{chiller_type}_temp_chiller']['calculated_total_load_indicator'] = total_load_indicator
        return chiller_predictions_output

    def _calculate_optimized_units(self, total_load, capacity_per_unit, max_units, min_load_ratio, max_load_ratio):
        """개선된 최적 가동 대수 계산 로직"""
        if pd.isna(total_load) or total_load <= 0 or capacity_per_unit <= 0:
            return 0

        # 1. 부하를 감당할 수 있는 이론적 최소 대수
        min_required_units = int(np.ceil(total_load / capacity_per_unit))
        min_required_units = min(max(0, min_required_units), max_units)

        if min_required_units == 0: # 부하가 거의 없을 때
            return 0

        best_units = min_required_units
        best_avg_load_ratio = 0 # 최적 부하율에 얼마나 가까운지를 나타내는 점수 (0이 최상)

        # 최소 필요 대수부터 최대 대수까지 탐색
        for num_op_units in range(min_required_units, max_units + 1):
            if num_op_units == 0: continue

            current_total_capacity = num_op_units * capacity_per_unit
            if current_total_capacity < total_load: # 현재 대수로 부하 감당 불가
                continue

            avg_load_per_unit_ratio = total_load / current_total_capacity

            # 부하율이 너무 낮거나 높은 경우는 피함 (효율 저하 또는 과부하 위험)
            if avg_load_per_unit_ratio < 0.1 and num_op_units > min_required_units : # 너무 낮은 부하율은 피함 (단, 최소필요대수일 경우는 제외)
                 continue
            if avg_load_per_unit_ratio > 1.0: # 과부하
                 continue


            # "최적 부하율 범위"에 얼마나 가까운지 또는 그 안에 있는지 평가
            # 여기서는 간단히 최적 범위 중간값과의 차이를 사용
            target_optimal_ratio = (min_load_ratio + max_load_ratio) / 2
            current_score = abs(avg_load_per_unit_ratio - target_optimal_ratio)

            if num_op_units == min_required_units: # 첫번째 유효한 대수는 일단 best로 설정
                best_units = num_op_units
                best_avg_load_ratio = current_score
            elif current_score < best_avg_load_ratio : # 더 나은 점수 (최적 부하율에 더 가까움)
                # 추가 조건: 부하율이 너무 낮아지는 것을 방지 (예: 3대를 20%로 돌리는 것보다 2대를 30%로 돌리는게 나을 수 있음)
                if avg_load_per_unit_ratio >= min_load_ratio * 0.8 : # 최소 부하율의 80% 이상은 되어야 함
                    best_units = num_op_units
                    best_avg_load_ratio = current_score
            # 만약 점수가 같을 경우, 더 적은 대수를 선호 (기본값)

        return best_units


    def optimize_chiller_operating_units(self, chiller_parameter_predictions):
        low_ch_load = chiller_parameter_predictions.get('low_temp_chiller', {}).get('calculated_total_load_indicator', 0)
        num_optimal_low = self._calculate_optimized_units(
            low_ch_load, self.rated_capacity_per_low_chiller, self.num_low_chillers,
            self.optimal_load_ratio_min, self.optimal_load_ratio_max
        )

        high_ch_load = chiller_parameter_predictions.get('high_temp_chiller', {}).get('calculated_total_load_indicator', 0)
        num_optimal_high = self._calculate_optimized_units(
            high_ch_load, self.rated_capacity_per_high_chiller, self.num_high_chillers,
            self.optimal_load_ratio_min, self.optimal_load_ratio_max
        )
        return {
            "optimal_low_temp_chillers_count": num_optimal_low,
            "optimal_high_temp_chillers_count": num_optimal_high
        }

    def optimize_cooling_tower_operating_units(self, optimal_chiller_counts_dict, chiller_parameter_predictions):
        # 냉동기 실제 부하를 기반으로 냉각탑 부하 계산
        low_ch_actual_load = chiller_parameter_predictions.get('low_temp_chiller', {}).get('calculated_total_load_indicator', 0)
        high_ch_actual_load = chiller_parameter_predictions.get('high_temp_chiller', {}).get('calculated_total_load_indicator', 0)

        # COP를 가정하여 열 방출량 추정 (Q_reject = Q_evap * (1 + 1/COP))
        # COP는 부하율, 냉각수 온도 등에 따라 변하지만, 여기서는 평균값 가정 (예: 3.5 ~ 4.5)
        assumed_cop = 4.0
        total_heat_rejection_load = 0
        if optimal_chiller_counts_dict["optimal_low_temp_chillers_count"] > 0 and pd.notna(low_ch_actual_load):
            total_heat_rejection_load += low_ch_actual_load * (1 + 1/assumed_cop)

        if optimal_chiller_counts_dict["optimal_high_temp_chillers_count"] > 0 and pd.notna(high_ch_actual_load):
            total_heat_rejection_load += high_ch_actual_load * (1 + 1/assumed_cop)

        num_optimal_ct = self._calculate_optimized_units(
            total_heat_rejection_load, self.rated_heat_rejection_per_ct, self.num_cooling_towers,
            self.optimal_load_ratio_min, self.optimal_load_ratio_max
        )
        return {"optimal_cooling_towers_count": num_optimal_ct}


# --- 메인 실행 로직 (현실성 있는 더미 데이터 생성 및 최적화 모델 개선) ---
if __name__ == '__main__':
    print("향상된 더미 데이터 생성 및 HVAC 최적화 시뮬레이션 시작...")

    # --- 1. 현실성 있는 더미 데이터 생성 (1년치, 1시간 간격) ---
    num_hours_in_year = 365 * 24
    datetime_index = pd.date_range(start='2023-01-01 00:00:00', periods=num_hours_in_year, freq='H')
    df_dummy_data = pd.DataFrame(index=datetime_index)

    # 시간 관련 피처 생성
    df_dummy_data['hour_of_day'] = df_dummy_data.index.hour
    df_dummy_data['day_of_year'] = df_dummy_data.index.dayofyear
    df_dummy_data['month'] = df_dummy_data.index.month

    # 외기 온도: 계절 + 일일 변화 + 노이즈
    base_temp_annual = 12  # 연평균 온도 (예: 서울)
    amplitude_annual = 15  # 연간 온도 진폭
    df_dummy_data['외기_온도'] = base_temp_annual + \
                             amplitude_annual * np.sin(2 * np.pi * (df_dummy_data['day_of_year'] - 90) / 365) + \
                             5 * np.sin(2 * np.pi * df_dummy_data['hour_of_day'] / 24) + \
                             np.random.normal(0, 1.5, num_hours_in_year) # 일일 변화 및 노이즈
    df_dummy_data['외기_온도'] = np.clip(df_dummy_data['외기_온도'], -15, 40) # 현실적인 범위

    # 외기 습도: 계절 + 일일 변화 (온도와 어느정도 반비례 또는 특정 계절에 높음) + 노이즈
    base_humidity_annual = 60 # 연평균 습도
    amplitude_humidity_annual = 25
    # 여름(6-8월)에 습도 높게, 겨울에 낮게
    month_effect_humidity = np.sin(2 * np.pi * (df_dummy_data['month'] - 4) / 12) # 7월경 최대
    df_dummy_data['외기_습도'] = base_humidity_annual + \
                              amplitude_humidity_annual * month_effect_humidity + \
                              10 * np.sin(2 * np.pi * df_dummy_data['hour_of_day'] / 24 + np.pi) + \
                              np.random.normal(0, 5, num_hours_in_year)
    df_dummy_data['외기_습도'] = np.clip(df_dummy_data['외기_습도'], 15, 99)

    # OAC (14대) 데이터 생성
    for i in range(1, 15):
        p = f"OAC{i}_"
        df_dummy_data[f'{p}FAN_Hz'] = np.random.uniform(35, 55, num_hours_in_year) + np.random.normal(0,2,num_hours_in_year)
        df_dummy_data[f'{p}FAN_Hz'] = np.clip(df_dummy_data[f'{p}FAN_Hz'], 30, 60)

        # Set points (외기 온도에 어느정도 반응)
        # 예열 SP: 외기가 추울수록 높아짐
        df_dummy_data[f'{p}예열_set_point'] = np.clip(20 - 0.3 * df_dummy_data['외기_온도'], 18, 25) + np.random.normal(0,0.5,num_hours_in_year)
        # 예냉 SP: 외기가 더울수록 낮아짐 (단, 냉각 SP보다는 높게)
        df_dummy_data[f'{p}예냉_set_point'] = np.clip(18 + 0.1 * df_dummy_data['외기_온도'], 16, 22) + np.random.normal(0,0.5,num_hours_in_year)
        # 냉각 SP: 외기가 더울수록 낮아짐
        df_dummy_data[f'{p}냉각_set_point'] = np.clip(15 + 0.05 * df_dummy_data['외기_온도'], 10, 18) + np.random.normal(0,0.5,num_hours_in_year)
        # 승온 SP: 실내 목표 온도 근처
        df_dummy_data[f'{p}승온_set_point'] = np.random.uniform(21, 24, num_hours_in_year) + np.random.normal(0,0.5,num_hours_in_year)
        df_dummy_data[f'{p}가습_set_point'] = np.random.uniform(40, 55, num_hours_in_year) # %RH

        # 개도율 (set point와 외기 조건, 다른 코일 상태에 따라 복잡하게 결정되나, 여기서는 단순화)
        # 외기 온도에 따른 주요 코일 개도율 경향성 부여
        df_dummy_data[f'{p}예열_개도율'] = np.clip(50 - 2.5 * df_dummy_data['외기_온도'] + np.random.normal(0,10,num_hours_in_year), 0, 100)
        df_dummy_data[f'{p}냉각_개도율'] = np.clip(2.0 * (df_dummy_data['외기_온도'] - 15) + np.random.normal(0,10,num_hours_in_year), 0, 100)
        df_dummy_data[f'{p}예냉_개도율'] = np.clip(df_dummy_data[f'{p}냉각_개도율'] * np.random.uniform(0.3, 0.7), 0, 100) # 냉각 부하의 일부
        df_dummy_data[f'{p}승온_개도율'] = np.clip(np.abs(df_dummy_data[f'{p}냉각_개도율'] - df_dummy_data[f'{p}예열_개도율'])*0.3 + np.random.normal(0,5,num_hours_in_year) , 0, 100) # 재열 개념 약간 반영
        df_dummy_data[f'{p}가습_개도율'] = np.clip(60 - df_dummy_data['외기_습도'] + np.random.normal(0,10,num_hours_in_year), 0, 100)

        # 후단 온/습도 (간단하게 setpoint 근처 또는 이전 단계 영향) - 모델 학습 시 사용하지 않는다면 생략 가능
        for coil_type in ['예열', '예냉', '냉각', '승온']:
             df_dummy_data[f'{p}{coil_type}_후단_온도'] = df_dummy_data[f'{p}{coil_type}_set_point'] + np.random.normal(0, 0.5, num_hours_in_year)
             df_dummy_data[f'{p}{coil_type}_후단_습도'] = df_dummy_data['외기_습도'] + np.random.normal(0, 5, num_hours_in_year) # 매우 단순화
             df_dummy_data[f'{p}{coil_type}_후단_습도'] = np.clip(df_dummy_data[f'{p}{coil_type}_후단_습도'],10,99)


    # 냉동기 데이터 (OAC 부하에 연동)
    total_cooling_load_indicator = pd.Series(0.0, index=df_dummy_data.index)
    total_precooling_load_indicator = pd.Series(0.0, index=df_dummy_data.index)
    for i in range(1, 15):
        p = f"OAC{i}_"
        total_cooling_load_indicator += df_dummy_data[f'{p}냉각_개도율'].fillna(0) * df_dummy_data[f'{p}FAN_Hz'].fillna(40)
        total_precooling_load_indicator += df_dummy_data[f'{p}예냉_개도율'].fillna(0) * df_dummy_data[f'{p}FAN_Hz'].fillna(40)

    for prefix, load_indicator_series in [("저온냉동기", total_cooling_load_indicator), ("고온냉동기", total_precooling_load_indicator)]:
        main_p = prefix + "_"
        # 부하 지표에 따라 메인 차압 개도율 변동 (0~100 스케일로 가정)
        base_opening = (load_indicator_series / load_indicator_series.max()) * 70 + 10 # 10~80% 범위로 스케일링
        df_dummy_data[f'{main_p}메인차압개도율'] = np.clip(base_opening + np.random.normal(0,5,num_hours_in_year), 5, 95)
        df_dummy_data[f'{main_p}서브차압개도율'] = np.clip(df_dummy_data[f'{main_p}메인차압개도율'] * np.random.uniform(0.7,1.1), 5, 95)
        df_dummy_data[f'{main_p}메인차압압력'] = df_dummy_data[f'{main_p}메인차압개도율'] * 0.015 + np.random.uniform(0.3,0.8) # 개도율과 압력 비례 가정
        # 기타 피처 (요청 목록에 있었던 것들)
        df_dummy_data[f'{main_p}메인supply압력'] = np.random.uniform(3, 7, num_hours_in_year)
        df_dummy_data[f'{main_p}메인supply온도'] = np.random.uniform(5, 9, num_hours_in_year) if "저온" in prefix else np.random.uniform(9, 13, num_hours_in_year)
        df_dummy_data[f'{main_p}메인return압력'] = df_dummy_data[f'{main_p}메인supply압력'] - np.random.uniform(0.5, 1.5, num_hours_in_year)
        df_dummy_data[f'{main_p}메인return온도'] = df_dummy_data[f'{main_p}메인supply온도'] + (df_dummy_data[f'{main_p}메인차압개도율']/100)*5 + np.random.uniform(1,3) # 부하 높을수록 온도차 커짐
        df_dummy_data[f'{main_p}메인return온도'] = np.clip(df_dummy_data[f'{main_p}메인return온도'], df_dummy_data[f'{main_p}메인supply온도']+0.5, 30)


    # 결측치 한번 더 확인 및 중앙값으로 채우기
    for col in df_dummy_data.columns:
        if df_dummy_data[col].isnull().any():
            df_dummy_data[col].fillna(df_dummy_data[col].median(), inplace=True)

    df_historical_data_improved = df_dummy_data.copy()
    print(f"향상된 더미 데이터 생성 완료: {df_historical_data_improved.shape[0]}개 행")
    # print(df_historical_data_improved[['외기_온도', '외기_습도', 'OAC1_냉각_개도율', '저온냉동기_메인차압개도율']].head())
    # print(df_historical_data_improved.isnull().sum())


    # --- 2. HVAC 옵티마이저 실행 ---
    hvac_optimizer = CleanroomHVACOptimizer()

    print("\n=== 모델 학습 시작 (향상된 더미 데이터 사용) ===")
    hvac_optimizer.train_oac_models(df_historical_data_improved)
    hvac_optimizer.train_chiller_models(df_historical_data_improved)
    print("=== 모델 학습 완료 ===\n")

    print("=== 예측 및 최적화 실행 (예시 시나리오) ===")
    current_outdoor_conditions = {'외기_온도': 28.0, '외기_습도': 70.0} # 여름철 예시

    example_oac_setpoints_input = []
    for i in range(1, hvac_optimizer.num_oacs + 1):
        oac_sp_prefix = f"OAC{i}_"
        example_oac_setpoints_input.append({
            f'{oac_sp_prefix}예열_set_point': 18.0, # 여름철 거의 사용 안함
            f'{oac_sp_prefix}예냉_set_point': 16.0,
            f'{oac_sp_prefix}냉각_set_point': 13.0, # 냉방 강하게
            f'{oac_sp_prefix}승온_set_point': 22.0, # 재열 최소화
            f'{oac_sp_prefix}가습_set_point': 50.0,
            f'{oac_sp_prefix}FAN_Hz': 50.0
        })

    print("\n--- OAC 코일 개도율 예측 ---")
    predicted_oac_coils = hvac_optimizer.predict_oac_coil_opening_ratios(
        current_outdoor_conditions, example_oac_setpoints_input
    )
    if 1 in predicted_oac_coils and predicted_oac_coils[1]:
        print("OAC 1 예측된 코일 개도율:")
        for coil_name, value in predicted_oac_coils[1].items():
            print(f"  {coil_name}: {value:.2f}%")

    print("\n--- 냉동기 파라미터 예측 ---")
    predicted_chiller_params = hvac_optimizer.predict_chiller_parameters(
        predicted_oac_coils, current_outdoor_conditions
    )
    print("예측된 냉동기 파라미터 및 총 부하 지표:")
    if predicted_chiller_params['low_temp_chiller']:
        print(f"  저온 냉동기: {predicted_chiller_params['low_temp_chiller']}")
    if predicted_chiller_params['high_temp_chiller']:
        print(f"  고온 냉동기: {predicted_chiller_params['high_temp_chiller']}")

    print("\n--- 냉동기 최적 가동 대수 산출 (개선된 로직) ---")
    optimal_chiller_units = hvac_optimizer.optimize_chiller_operating_units(predicted_chiller_params)
    print(f"  최적 저온 냉동기 가동 대수: {optimal_chiller_units['optimal_low_temp_chillers_count']} 대")
    print(f"  최적 고온 냉동기 가동 대수: {optimal_chiller_units['optimal_high_temp_chillers_count']} 대")

    print("\n--- 냉각탑 최적 가동 대수 산출 (개선된 로직) ---")
    optimal_ct_units = hvac_optimizer.optimize_cooling_tower_operating_units(optimal_chiller_units, predicted_chiller_params)
    print(f"  최적 냉각탑 가동 대수: {optimal_ct_units['optimal_cooling_towers_count']} 대")
    print("\n=== 예측 및 최적화 완료 ===")

향상된 더미 데이터 생성 및 HVAC 최적화 시뮬레이션 시작...


<ipython-input-1-7f38a0e55205>:357: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  datetime_index = pd.date_range(start='2023-01-01 00:00:00', periods=num_hours_in_year, freq='H')
<ipython-input-1-7f38a0e55205>:388: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_dummy_data[f'{p}FAN_Hz'] = np.random.uniform(35, 55, num_hours_in_year) + np.random.normal(0,2,num_hours_in_year)
<ipython-input-1-7f38a0e55205>:393: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_dummy_data[f'{p}예

향상된 더미 데이터 생성 완료: 8760개 행

=== 모델 학습 시작 (향상된 더미 데이터 사용) ===
OAC 모델 학습 시작...
